### List of Imports

In [8]:
import pandas as pd
import numpy as np
import requests
import pandas as pd
from Bio.PDB import Selection, NeighborSearch
from collections import defaultdict
#

# Class object that we utilize as framework

In [9]:
class MutationalClusterer:
    def __init__(self, work_dir, logging=True):
        '''
        General File structure should be something like:
        https://tree.nathanfriend.io/?s=(%27opGHs!(%27fancy8~fullPath!fJse~trailingSlash8~rootDot8)~K(%27K%27work_dir5C0QD64Swissprot.DB4OthA_files_for_Mlookup_in_d6.DB--IntAmediates4F_protein_mulGple_seq2.Ji4summary_staGsGcs29quJity2_df93-Results4*Mdf9*MplotsL*4*additHJ4*5Pymol_output-Example4screenshotL35PDBsQ4D6_parsA_s7%204NeighbFhood_clustAA.py43-3-%27)~vAsiH!%271%27)*%20%20-5*0HsAvaGH2_Jignment3...%2F4-*5%5Cn*6atabase7cripts8!true9.csv4AerFourGtiHonJalKsFce!L.png4Mc0_Q-S7-%01QMLKJHGFA987654320-*
        .
        └── work_dir/
            ├── Conservation/
            │   ├── Scripts
            │   ├── Database/
            │   │   ├── Swissprot.DB
            │   │   └── Other_files_for_conservation_lookup_in_database.DB
            │   ├── Intermediates/
            │   │   ├── our_protein_multiple_seq_alignment.ali
            │   │   ├── summary_statistics_alignment.csv
            │   │   ├── quality_alignment_df.csv
            │   │   └── .../
            │   └── Results/
            │       ├── conservation_df.csv
            │       ├── conservation_plots.png
            │       └── additonal
            ├── Pymol_output/
            │   └── Example/
            │       ├── screenshot.png
            │       └── .../
            └── PDBs/
                ├── Scripts/
                │   ├── Database_parser_scripts 
                │   ├── Neighbourhood_clusterer.py
                │   └── .../
                └── .../
        
        '''
        self.work_dir = work_dir #the directory we want to work in
        self.log_dir = logging # we want to store log results for whatever we do.
        self.gene_name = None
        #Your other features of the class that we need.
        
        

    def _visualize_clusters_pymol(self, pdb:str) -> None:
        """
        Utility function to visualize cluster in pymol.
        """
        
        #from pymol import cmd
        pass

    def _plot_clusters(self, pdb:str) -> object:
        """
        Helper function to plot some statistics or quick interactive plots to investigate clustering.
        Mostly thought about pyplot or plotly interactive plots i.e alignment where we can see the conservation etc.
        https://plotly.com/python/alignment-chart/
        """


    def compute_neighbours(self, pdb:str, cutoff=8.0) -> pd.DataFrame:

        # Initialize parser and retrieve structure
        structure = parser.get_structure(protname, pdbfile1)
        atom_list = Selection.unfold_entities(structure, "A")  # Retrieve all atoms
    
        # Initialize NeighborSearch with all atoms and prepare to store results
        ns = NeighborSearch(atom_list)
        neighbor_dict = defaultdict(set)  # Use set to avoid duplicates
    
        # Define list of standard amino acids to exclude solvents and ligands
        aa_lst = [
            "VAL", "ALA", "GLY", "TRP", "ARG", "LYS", "LEU", "ILE", "ASP", "ASN",
            "GLN", "GLU", "PRO", "TYR", "PHE", "SER", "THR", "CYS", "MET", "HIS"
        ]
    
        # Search for neighboring residues for each atom
        for atom in atom_list:
            residue = atom.get_parent()
            res_name = residue.get_resname()
            res_id = residue.get_id()[1]
    
            # Skip non-amino acid residues
            if res_name not in aa_lst:
                continue
    
            # Search for neighboring residues within the cutoff distance
            for neighbour in ns.search(atom.get_coord(), cutoff, "R"):
                neighbour_id = neighbour.get_id()[1]
                if neighbour_id != res_id:  # Exclude the residue itself
                    neighbour_dict[res_name + str(res_id)].add(neighbour_id)
    
            
        # Convert the neighbor dictionary to a list of tuples
        neighbour_data = [(res_id, ' '.join(map(str, sorted(neighbours)))) for res_id, neighbours in neighbour_dict.items()]

        # Create a pandas DataFrame
        df_neighbours = pd.DataFrame(neighbour_data, columns=['Residue_ID', 'Neighbours'])
        return df_neighbours
    

    def conservation(self, uniprot_id):
        '''Gets 3 different types of Conservation:
        - Shannon conservation: 
        Shannon entropy. 
        Higher values indicate lower conservation and greater variability at the site.
        
        - Relative conservation:
        Kullback-Leibler divergence.
        Higher values indicate greater conservation and lower variability at the site.
        
        - Lockless conservation
        Evolutionary conservation parameter defined by Lockless and Ranganathan (1999). 
        Higher values indicate greater conservation and lower variability at the site.
        '''

        if self.log_dir and not os.path.exists(self.log_dir):
            os.makedirs(self.log_dir)
        
        mmseq_fasta_result = self._mmseq_multi_fasta(uniprot_id=uniprot_id, outdir=self.work_dir)
        #get 3 different conservation scores in a pandas df.
        conserv_df = self._get_conservation(path_to_msa=mmseq_fasta_result)
        self.conservation_df = conserv_df

        conserv_df.to_csv(f"{self.log_dir}/conservation_df.csv")
        
    def _mmseq_multi_fasta(self, uniprot_id:str, outdir:str, 
                      sensitivity=7, filter_msa=0,
                     query_id = 0.6):
        """
        uniprot_id: The unique uniprot identifier used to fetch the corresponding fasta file that will be used as a template for mmseq2
        outdir: location where result files will be stored.
        sensitivity: mmseq2 specific parameter that goes from 1-7. The higher the more sensitive the search.
        filter_msa = 0 default. if 1 hits are stricter.
        query_id = 0.6 [0, 1]  the higher the more identity with query is retrieved. 1 means ONLY the query hits while 0 means take everything possible.
        """

        #we blast with this fasta as query.
        trgt_fasta_seq = self._get_gene_fasta(uniprot_id)
        #Make outdir for all required files.
        #we need to write it out to file.
        with open(f"{self.work_dir}/{uniprot_id}_fasta.fa", "w") as fasta_out:
            fasta_out.write(f">{uniprot_id}\n")
            fasta_out.write(trgt_fasta_seq)

        #fetch pre downloaded database from a parent folder.
        msa_file = None
        new_location = None
        try:
            DB_storage_location = f"{work_dir}"
            #shutil.copy(previous_path, savepath)
            bash_curl_cmd = f"mmseqs createdb {self.work_dir}/{uniprot_id}_fasta.fa {DB_storage_location}/query_fastaDB" 
            bash_curl_cmd_rdy = bash_curl_cmd.split()
            #run first cmd which setups query database based on our input fasta file
            result_setup_query_db = run(bash_curl_cmd_rdy, stdout=PIPE, stderr=PIPE, 
                                 universal_newlines=True)
            bash_curl_cmd_2 = f"mmseqs search {DB_storage_location}/query_fastaDB {DB_storage_location}/swiss_DB {DB_storage_location}/result_DB {DB_storage_location}/tmp -s {sensitivity}"    
            bash_curl_cmd_rdy_2 = bash_curl_cmd_2.split()
            #run 2nd cmd which blasts against swiss_DB and generates the resultDB (i.e our hits that were found)
            result_setup_blast_db = run(bash_curl_cmd_rdy_2, stdout=PIPE, stderr=PIPE, 
                                 universal_newlines=True)
            #mmseqs convert2fasta DB_clu_rep DB_clu_rep.fasta
            bash_curl_cmd_5 = f"mmseqs result2msa {DB_storage_location}/query_fastaDB {DB_storage_location}/swiss_DB {DB_storage_location}/result_DB {DB_storage_location}/{uniprot_id}_out.fasta --msa-format-mode 3 --filter-msa {filter_msa} --qid {query_id}" 
            bash_curl_cmd_5_rdy = bash_curl_cmd_5.split()
            result_setup_msa_convert = run(bash_curl_cmd_5_rdy, stdout=PIPE, stderr=PIPE, 
                                 universal_newlines=True)
            #delete last line.. required.
            sed_cmd = f'sed -e 1,4d -e $d {DB_storage_location}/{uniprot_id}_out.fasta'        
            bash_curl_cmd_6_rdy = sed_cmd.split()
            #f"{DB_storage_location}/{uniprot_id}_new_out.fasta"
            with open(f"{DB_storage_location}/{uniprot_id}_new_out.fasta", "w") as new_fasta:
                result_truncation = run(bash_curl_cmd_6_rdy, stdout=new_fasta, stderr=PIPE, 
                                 universal_newlines=True)
            # Specify the path to your MSA file
            msa_file = f"{DB_storage_location}/{uniprot_id}_new_out.fasta"
            #transfer the meta file to another location and delete useless files.
            # we need to delete : all uniprot* files. 
            # all query*. All result* 
            new_location = f"{self.work_dir}/{uniprot_id}.fasta"
            shutil.copy(msa_file, new_location)
            #remove_files_and_dirs_msa(DB_storage_location, uniprot_id=uniprot_id)
            
        except Exception as error:
            print(error)
        #we want the path to msa_file for downstream analysis.
        return new_location

    def _get_gene_fasta(self, uniprot_id:str):
        '''
        Helper function to grab the sequence 
        based on the Uniprot ID
        '''
        fields = "sequence"
        URL = f"https://rest.uniprot.org/uniprotkb/search?format=fasta&fields={fields}&query={uniprot_id}"
        resp = self._get_url(URL)
        resp = resp.iter_lines(decode_unicode=True)
        seq = ""
        i = 0
        for lines in resp:
            if i > 0:
                seq += lines
            i += 1
        return seq

    def _get_conservation(self, path_to_msa:str):    
        '''
        Helper function to compute 3 different types of conservation.
        
        - Shannon conservation: 
        Shannon entropy. 
        Higher values indicate lower conservation and greater variability at the site.
        
        - Relative conservation:
        Kullback-Leibler divergence.
        Higher values indicate greater conservation and lower variability at the site.
        
        - Lockless conservation
        Evolutionary conservation parameter defined by Lockless and Ranganathan (1999). 
        Higher values indicate greater conservation and lower variability at the site.
        '''
        canal = Canal(fastafile=path_to_msa, #Multiple sequence alignment (MSA) of homologous sequences
          ref=0, #Position of reference sequence in MSA, use first sequence always
          startcount=0, # ALways 0 because our seqs are always from 1 - end
          verbose=False) # no verbosity 
    
        result_cons = canal.analysis(method="all")
        return result_cons

    def _get_url(self, url):
        '''Helper function that uses requests for Downloads.'''
        try:
            response = requests.get(url)  
            if not response.ok:
                print(response.text)
        except:
            response.raise_for_status()
            #sys.exit() 
        return response

    def _get_cosmic_mutations(self, uniprot_id:str, path_to_genome_screen_tsv:str, **kwargs)->pd.DataFrame:

        #we convert the uniprot id to gene name for fetch and store it in self.gene_name so we dont need to update it for other databases.
        self.gene_name = self.__get_gene_name(uniprot_id)
        
        
        #default columns we want to retrieve. can be changed / added through kwargs later
        usecols=['GENE_SYMBOL',
         'MUTATION_AA', 'MUTATION_DESCRIPTION',
       'MUTATION_ZYGOSITY', 'LOH', 'CHROMOSOME', 
                 'GENOME_START', 'GENOME_STOP']

        df = dd.read_csv(path, sep="\t", dtype={'CHROMOSOME': 'object',
       'MUTATION_ZYGOSITY': 'object', 'GENOME_START': 'float64',
       'GENOME_STOP': 'float64',
       'LOH': 'object'}, usecols=usecols)  #specify dtype / usecols to minimize memory usage required through load in.


        #we need to switch these tuples and then map the 1letter aa code to 3letter aa 
        #for later compatibility.
        lst =  [('Val',"V"), ('Ile',"I"), ('Leu',"L"), ('Glu',"E"), ('Gln',"Q"),
                    ('Asp',"D"), ('Asn',"N"), ('His',"H"), ('Trp',"W"), ('Phe',"F"), ('Tyr',"Y"), 
                    ('Arg',"R"), ('Lys',"K"), ('Ser',"S"), ('Thr',"T"), ('Met',"M"), ('Ala',"A"), 
                    ('Gly',"G"), ('Pro',"P"), ('Cys',"C")]
        
        lst = [(y, x) for x, y in lst] #switch y and x position for convinience

        canonical_aas = defaultdict(lambda: "X", lst) #default if key not found = "X"

        #filtering based on "missense" mutation. this can be tricky and sometimes messy but lets stick with that
        df_re = df[df["MUTATION_DESCRIPTION"].str.contains("missense")]

        #now lets filter our uniprot gene name
        df_re = df_re[df_re["GENE_SYMBOL"] == f"{gene_name}"]

        #retrieve relevant information
        meta = ('Gene name', 'str') 
        df_re['CHROMOSOME'] = df_re['CHROMOSOME'].astype('object')
        df_re['WT_AA'] = df_re['MUTATION_AA'].str[2].apply(lambda x: canonical_aas[x], meta=meta)
        df_re['MUTATION_POSITION'] = df_re['MUTATION_AA'].str[3:-1]
        df_re['MUTATED_AA'] = df_re['MUTATION_AA'].str[-1].apply(lambda x: canonical_aas[x], meta=meta)

        #redundant so we drop it
        df_re = df_re.drop("MUTATION_AA", axis=1)

        #now we use compute() which finally does the computation (before all actions were "lazy" computations
        # so we dont actually need the RAM. now we do it though.)
        cosmic_df = df_re.compute()
    
        cosmic_df["GENOME_START"] = cosmic_df["GENOME_START"].astype(int)
        cosmic_df["GENOME_STOP"] = cosmic_df["GENOME_STOP"].astype(int)

        #return df
        return cosmic_df


    def _get_gnomad_mutations(self, gene_name:str, gnomad_data_table_path:str, **kwargs)-> pd.Dataframe:
        """Documentation.
        Currently this part does not convert the result to a df.
        I will implement it and return a pandas DF
        """
        mt = hl.read_matrix_table(path)  #matrix table because df would not work with such large data.


        #string based search because there is NO API for gnomAD.
        substring1 = Gene_name
        substring2 = "missense"
        
        mt = mt.annotate_rows(Gene_names=mt.info.vep.map(
            lambda x: x.split("\|")[3]) ,
                          type_of_change = mt.info.vep.map(
            lambda x: x.split("\|")[1]) , 
                          AA_change = mt.info.vep.map(
            lambda x: x.split("\|")[11]) , 
                          ENST_identifier= mt.info.vep.map(
            lambda x: x.split("\|")[6])
    
        ) 
                 
        filtered_mt_2 = mt.filter_rows(
        #hl.any(lambda x: hl.str(x).contains(substring3), mt.AA_change)
        hl.any(lambda x: hl.str(x).contains(substring1), mt.info.vep) &
        hl.any(lambda x: hl.str(x).contains(substring2), mt.info.vep)
        
        )
                         
        filtered_mt_3 = filtered_mt_2.annotate_rows(
            Allele_count_int = filtered_mt_2.info.AC,
            Allele_frequency_float = filtered_mt_2.info.AF,
            Allele_number_int = filtered_mt_2.info.AN,
            Gene_name_str = _replace_empty(filtered_mt_2.Gene_names), 
            Mutation_change_str = _replace_empty(filtered_mt_2.AA_change),
            Type_of_change_str = _replace_empty(filtered_mt_2.type_of_change))
        
        #this can be again regulated later trough kwargs**
        rows_to_keep = ["Gene_name_str", "Mutation_change_str", "Type_of_change_str", "Allele_count_int",
                    "Allele_frequency_float", "Allele_number_int"]
    
    
        selected_rows = filtered_mt_3.select_rows(
            Allele_count_int=filtered_mt_3.Allele_count_int,
            Allele_frequency_float=filtered_mt_3.Allele_frequency_float,
            Allele_number_int=filtered_mt_3.Allele_number_int,
            Gene_name_str=hl.str(filtered_mt_3.Gene_name_str),
            Mutation_change_str=hl.str(filtered_mt_3.Mutation_change_str),
            Type_of_change_str=hl.str(filtered_mt_3.Type_of_change_str)
                )
    
        save_buffer = selected_rows.select_rows(*rows_to_keep)
        select_rows_out = save_buffer.rows()

        #part missing to convert to pandas DF.
        
        #return pd.Dataframe() 

    def _map_clinvar(self, gene_name:str, clinvar_variant_summary_txt:str, **kwargs)-> pd.Dataframe:
        """
        Documentation missing.
        """

        #can be regulated through kwargs
        use_cols = ["Type", "Name", "GeneSymbol",
           "ClinicalSignificance", "PhenotypeList",
           "Assembly", "ChromosomeAccession", 
           "Chromosome", "Start", "Stop"]
        
        #this here as well / mapping needed to save memory at load in.
        column_data_types = {
        "Type": str,
        "Name": str,
        "GeneSymbol": str,
        "ClinicalSignificance": str,
        "PhenotypeList": str,
        "Assembly": str,
        "ChromosomeAccession": str,
        "Chromosome": str,
        "Start": int,
        "Stop": int
        }

        #lets read in the clinvar all var file.
        df_work = pd.read_csv(clinvar_variant_summary_txt, sep="\t", usecols=use_cols, dtype=column_data_types)
    
        df_work.loc[:, "AA_change"] = df_work["Name"].str.split().str.get(-1)
        df_work.loc[:, "AA_change"] = df_work["AA_change"].str.replace("(", "")
        df_work.loc[:, "AA_change"] = df_work["AA_change"].str.replace(")", "")
        
        df_work.loc[:,"Original_AA"] = df_work["AA_change"].str[2:5]
        df_work.loc[:,"Modified_AA"] = df_work["AA_change"].str[-3:]
        df_work['Position'] = pd.to_numeric(df_work['AA_change'].str[5:-3], errors='coerce')
        
        # Drop rows with NaN values in the 'Position' column
        df_work.dropna(subset=['Position'], inplace=True)
        df_work['Position'] = df_work['Position'].astype(int)
        
        df_work["Genomic_location"] = df_work["Chromosome"] + ":" + df_work["Start"].astype(str)
        df_work["gnomad_aa_change"] = "p." + df_work["Original_AA"] + df_work["Position"].astype(str) + df_work["Modified_AA"]
        
        df_work = df_work.drop("AA_change", axis=1)
        df_work = df_work.drop("Name", axis=1)
        df_work = df_work.drop("Chromosome", axis=1)
        df_work = df_work.drop("Start", axis=1)
        df_work = df_work.drop("Stop", axis=1)
        
        df_work["Allele_count"] = [np.nan] * len(df_work)
        df_work["Allele_number"] = [np.nan] * len(df_work)
        df_work["Allele_frequency"] = [np.nan] * len(df_work)
        
        
        accepted_residues = ["Ala", "Gly", "Ser", "Leu", "Pro",
                        "Ile", "Val", "Phe", "Tyr", "Trp",
                         "His", "Thr", "Asn", "Gln", "Asp", 
                         "Glu","Cys", "Met", "Lys", "Arg"]
        
        #filtering based on our Gene name.
        df_clinvar = df_work[(df_work["Type"] == "single nucleotide variant") & 
            (df_work["GeneSymbol"] == Gene_name) &
            (df_work["Assembly"] == "GRCh37") & 
            (df_work['Original_AA'].isin(accepted_residues)) &
            (df_work['Modified_AA'].isin(accepted_residues)) ]
        
        return df_clinvar
        

In [ ]:
# Initialize variables to avoid errors
    updated_clinvar_df = cbioport_df = cosmic_df = gnomad_df = clinvar_df = gnomad_mut_dict = gnomad_mutation_dict = None

    
    # Step 1: Cosmic mutations
    try:
        cosmic_df = get_cosmic_mutations(gene_name=main_prot_name)
    except Exception as error:
        print(error)

    #we save it in the folder for the protein outside of monomer / pos at the base level. 
    save_dataframe_to_csv(cosmic_df, path, "cosmic_mutations")

    # Step 2: Map gnomad mutations
    try:
        gnomad_table_path = map_gnomad(Gene_name=main_prot_name, outpath=oligo_state_to_check)
        gnomad_df, gnomad_mutation_dict = gnomad_to_pandas(Gene_name=main_prot_name, path_to_tsv=gnomad_table_path, fasta_seq=main_prot_seq)
    except Exception as error:
        print(error)
    
    save_dataframe_to_csv(gnomad_df, path, "gnomad_mutations")

    # Step 3: Gather mutations from clinvar
    try:
        clinvar_df = map_clinvar(Gene_name=main_prot_name)
        clinvar_map_outpath = f"{path}/clinvar_intermediate.csv"
    except Exception as error:
        print(error)
    
    save_dataframe_to_csv(clinvar_df, path, "clinvar_intermediate")

    # Step 4: Map clinvar to gnomad
    try:
        list_to_be_searched, clinvar_df = map_clinvar_to_gnomad_1(Gene_name=main_prot_name, clinvar_df=clinvar_df,
                                                                  gnomad_mut_dict=gnomad_mutation_dict, clinvar_mapped_df_path=clinvar_map_outpath)
    except Exception as error:
        print(error)
        # Update clinvar muts that were found 1 step before.
        try:
            updated_clinvar_df = update_clinvar_muts_based_on_gnomad(clinvar_df=clinvar_df, gnomad_dict=gnomad_mutation_dict)
        except Exception as error:
            print(error)
    
    save_dataframe_to_csv(updated_clinvar_df, path, "clinvar_mutations")

    # Step 5: Fetch additional info from cbioportal
    try:
        gene_name = get_hugo_name(uniprot_id)
        print(f"This is gene name in hugo: {gene_name}")
        cbioport_df = get_cbioportal_info(gene_name=gene_name)
        save_dataframe_to_csv(cbioport_df, path, "cbioport_mutations")
    except Exception as error:
        print(error)
    
    # Print shapes (if available)
    dataframes = [cosmic_df, updated_clinvar_df, gnomad_df, cbioport_df]
    for df in dataframes:
        try:
            print(f"This is df shape: {df.shape}")
        except Exception as error:
            print(error)